# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
# pattern = ['STOCH', 'RSI', 'LinearReg']
work_timeframe = '1h'
higher_timeframe = '4h'
opt_limit = 300
load = True

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [20, 25, 30, 35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [15, 20, 25]},
                'LinearReg': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h/4h, trade type is buy
Number of combinations is 12


  0%|                                                                                                                                                                           | 0/12 [00:00<?, ?it/s]


Load the datasets...
Binance
ByBit
BinanceFutures


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [03:36<00:00, 18.02s/it]


# Check local statistics

In [2]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
0,STOCH_RSI,14,20,9,7,3,15,0.5543,0.1298,0.3461,0.3461,0.3461,0.5369,0.6081,0.6209,0.7430,0.7557,0.7634,0.7863,0.7863,0.7863,0.7863,0.8117,0.9237,0.9237,0.9237,0.9237,0.9237,0.9237,0.9542,0.9746,1.24,-0.87,3.30,1.14,2.87,5.13,5.81,5.93,7.09,7.21,7.29,7.51,7.17,6.87,7.00,7.75,8.82,8.79,8.82,8.82,8.74,8.79,9.11,9.30,1,0.715762,6.401250,-0.284238,6.401250
11,STOCH_RSI,14,35,9,7,3,25,1.2028,1.0823,0.9583,0.7417,0.6983,0.7339,0.7563,0.7806,0.7860,0.8175,0.8915,0.8909,0.8790,0.9461,0.9736,1.0045,1.0089,1.0363,1.0942,1.1703,1.2182,1.3369,1.4082,1.4812,0.29,0.30,0.36,0.38,0.21,0.09,0.00,-0.08,-0.03,-0.04,-0.03,0.03,-0.09,0.13,0.32,0.52,0.52,0.46,0.69,0.98,1.19,1.43,1.55,1.61,102,0.995729,0.449583,-0.435625,45.857500
1,STOCH_RSI,14,20,9,7,3,20,0.2616,0.1239,0.2848,0.2848,0.2848,0.5796,0.7139,0.7234,0.8142,0.8237,0.8294,0.8464,0.8464,0.8464,0.8464,0.8872,1.0234,1.0421,1.0679,1.0679,1.0679,1.0679,1.0906,1.1058,1.04,0.28,1.05,-1.71,1.00,7.25,9.71,9.03,7.57,9.25,8.71,7.99,8.41,7.66,7.65,11.25,13.16,13.64,14.32,13.81,12.48,10.82,10.13,9.24,2,0.772100,8.072500,-0.455800,16.145000
2,STOCH_RSI,14,20,9,7,3,25,0.2616,0.1239,0.2848,0.2848,0.2848,0.5796,0.7139,0.7234,0.8142,0.8237,0.8294,0.8464,0.8464,0.8464,0.8464,0.8872,1.0234,1.0421,1.0679,1.0679,1.0679,1.0679,1.0906,1.1058,1.04,0.28,1.05,-1.71,1.00,7.25,9.71,9.03,7.57,9.25,8.71,7.99,8.41,7.66,7.65,11.25,13.16,13.64,14.32,13.81,12.48,10.82,10.13,9.24,2,0.772100,8.072500,-0.455800,16.145000
3,STOCH_RSI,14,25,9,7,3,15,1.8614,0.7297,0.8538,0.6330,0.3806,0.4130,0.4172,0.4123,0.4391,0.4223,0.4186,0.4246,0.4246,0.4246,0.4246,0.4312,0.4455,0.4274,0.4628,0.4815,0.4994,0.5174,0.5247,0.5296,0.43,0.39,0.59,0.46,-0.17,-0.72,-0.83,-0.59,-0.70,-0.75,-0.75,-0.79,-0.77,-0.30,-0.41,-0.49,-0.45,-0.38,-0.57,-0.70,-0.42,-0.49,-0.45,-0.45,6,0.541621,-0.387917,-2.750275,-2.327500
6,STOCH_RSI,14,30,9,7,3,15,1.4341,0.8800,0.8413,0.6229,0.4114,0.3991,0.3707,0.4040,0.4133,0.4021,0.4181,0.4468,0.4401,0.5336,0.7119,0.7814,0.7679,0.7506,0.8008,0.8358,0.8862,0.9962,1.0351,1.0553,0.32,0.23,0.00,0.00,-0.20,-0.25,-0.43,-0.25,-0.34,-0.29,-0.18,-0.35,-0.13,0.24,1.26,1.18,0.68,-0.06,0.19,1.00,1.15,0.87,0.75,0.87,13,0.693279,0.260833,-3.987371,3.390833
9,STOCH_RSI,14,35,9,7,3,15,1.2481,1.0646,1.0060,0.7379,0.5549,0.5351,0.5364,0.5811,0.5790,0.5740,0.5777,0.5813,0.5594,0.6350,0.7465,0.7878,0.7650,0.8244,0.8797,0.9178,0.9533,1.1472,1.2642,1.3312,0.31,0.23,0.17,0.00,-0.18,-0.06,-0.38,-0.11,-0.24,-0.25,-0.18,-0.33,-0.13,0.13,0.19,0.16,0.58,0.25,0.58,0.55,1.13,1.07,1.47,1.22,27,0.807817,0.257500,-5.188950,6.952500
10,STOCH_RSI,14,35,9,7,3,20,1.1991,1.1279,0.9399,0.7408,0.6693,0.7032,0.7111,0.7393,0.7363,0.7540,0.7673,0.7605,0.7448,0.8185,0.8603,0.8990,0.9003,0.9330,0.9806,1.0309,1.0586,1.1716,1.2449,1.2995,0.28,0.30,0.39,0.37,0.15,0.09,0.00,-0.08,-0.07,-0.14,-0.03,-0.05,-0.11,0.12,0.24,0.42,0.39,0.41,0.57,0.83,1.14,1.13,1.20,1.41,74,0.907946,0.373333,-6.812008,27.626667
5,STOCH_RSI,14,25,9,7,3,25,0.8146,0.4568,0.3654,0.3052,0.2775,0.3281,0.3402,0.3566,0.3672,0.3534,0.3751,0.3686,0.3702,0.3719,0.3781,0.3906,0.4041,0.4031,0.4210,0.4288,0.4378,0.4485,0.4582,0.4720,0.43,0.19,0.55,0.23,-0.12,-0.20,-

In [2]:
# 1h/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
11,STOCH_RSI,14,35,9,7,3,25,2.7930,2.5515,3.1857,4.3789,6.9252,8.5621,8.7646,9.0394,9.1694,9.3102,9.3246,9.3246,9.3246,9.3246,8.4069,7.6444,7.6444,7.6593,7.6847,7.6962,7.6962,8.3895,9.0073,9.2087,1.29,1.43,1.71,2.67,4.80,6.62,6.86,6.87,6.75,6.91,5.81,5.72,5.79,5.13,4.47,4.41,4.38,4.74,6.36,5.09,5.05,5.95,7.37,7.32,10,7.625667,5.145833,66.256667,51.458333
10,STOCH_RSI,14,35,9,7,3,20,4.2443,6.0905,5.0884,7.0736,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,1.29,1.76,1.71,2.67,4.80,6.62,6.99,6.87,6.75,6.93,6.17,5.72,6.20,5.59,4.75,4.47,4.38,4.74,6.36,5.68,5.05,5.95,7.47,7.32,8,9.270700,5.260000,66.165600,42.080000
9,STOCH_RSI,14,35,9,7,3,15,2.5456,4.4866,7.1517,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,10.0000,0.77,1.43,1.25,2.79,5.54,6.62,7.03,6.87,6.62,6.78,6.17,5.56,6.20,5.68,4.95,4.79,4.48,5.34,6.38,5.68,5.95,6.51,7.08,7.51,4,9.340996,5.332500,33.363983,21.330000
4,STOCH_RSI,14,25,9,7,3,20,10.0000,10.0000,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,10.48,12.65,6.09,0.79,3.90,-0.36,-1.92,3.36,9.88,8.38,5.56,6.47,2.28,2.30,1.39,-1.07,-8.27,-0.95,-0.95,-2.89,0.49,2.35,1.77,-1.69,1,1.937988,2.501667,0.937988,2.501667
7,STOCH_RSI,14,30,9,7,3,20,10.0000,10.0000,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,1.3996,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,0.9241,10.48,12.65,6.09,0.79,3.90,-0.36,-1.92,3.36,9.88,8.38,5.56,6.47,2.28,2.30,1.39,-1.07,-8.27,-0.95,-0.95,-2.89,0.49,2.35,1.77,-1.69,1,1.937988,2.501667,0.937988,2.501667
5,STOCH_RSI,14,25,9,7,3,25,2.3025,0.8851,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5075,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,6.82,4.90,-0.81,-2.03,-2.43,-5.27,-3.64,1.99,4.56,2.52,1.53,0.06,-2.02,-2.43,-4.01,-8.54,-8.79,-5.13,-6.02,-5.44,-2.90,-2.23,-4.10,-5.51,2,0.616750,-2.038333,-0.766500,-4.076667
8,STOCH_RSI,14,30,9,7,3,25,2.3025,0.8851,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5937,0.5075,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,0.4425,6.82,4.90,-0.81,-2.03,-2.43,-5.27,-3.64,1.99,4.56,2.52,1.53,0.06,-2.02,-2.43,-4.01,-8.54,-8.79,-5.13,-6.02,-5.44,-2.90,-2.23,-4.10,-5.51,2,0.616750,-2.038333,-0.766500,-4.076667
0,STOCH_RSI,14,20,9,7,3,15,0.7901,0.4752,0.2675,0.2136,0.2136,0.2108,0.2072,0.1506,0.1184,0.1184,0.1184,0.1184,0.1329,0.1475,0.1475,0.1475,0.1475,0.1475,0.2175,0.2879,0.2879,0.2879,0.2879,0.2879,0.18,0.11,-0.11,-0.30,-0.36,-0.24,-0.13,-0.38,-0.87,-1.03,-0.64,-0.32,-0.04,0.15,0.02,0.00,-0.07,-0.29,0.04,0.29,0.05,-0.05,-0.12,-0.10,2,0.230400,-0.175417,-1.539200,-0.350833
1,STOCH_RSI,14,20,9,7,3,20,0.7901,0.4752,0.2675,0.2136,0.2136,0.2108,0.2072,0.1506,0.1184,0.1184,0.1184,0.1184,0.1329,0.1475,0.1475,0.1475,0.1475,0.1475,0.2175,0.2879,0.2879,0.2879,0.28

In [2]:
# 4h/1d
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
54,STOCH_RSI_LinearReg,14,35,9,7,3,20,12,0,0.9046,0.8223,0.8012,0.8108,0.9514,1.1802,1.1703,1.1583,1.1686,1.2966,1.2792,1.2433,1.2172,1.4073,1.6835,1.7170,1.7037,1.5505,1.5205,1.4401,1.2942,1.2387,1.3125,1.3125,0.94,0.80,1.06,0.27,0.28,0.61,0.62,0.57,0.48,2.05,1.54,0.99,1.16,0.89,0.72,-0.14,0.32,0.87,1.04,-0.02,-0.46,-1.01,-1.10,-1.26,20,1.257688,0.467500,5.153750,9.350000
57,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,2.1024,1.8089,1.0145,1.0355,1.1357,1.1146,1.1760,1.1311,1.1089,1.2111,1.3024,1.3662,1.3646,1.3520,1.4679,1.5827,1.6096,1.3661,1.3569,1.2847,1.0595,0.9722,1.1148,1.1148,1.47,0.90,1.06,0.79,1.45,0.70,0.18,0.12,0.45,1.79,0.95,0.70,1.43,-0.50,2.16,1.47,1.06,2.28,2.64,1.69,-0.20,-0.67,-1.63,-2.55,10,1.298046,0.739167,2.980458,7.391667
52,STOCH_RSI_LinearReg,14,35,9,7,3,20,8,0,2.1871,1.9546,1.0397,1.0619,1.1543,1.1467,1.2088,1.1597,1.1355,1.2128,1.3530,1.4198,1.3798,1.3661,1.4863,1.6064,1.6064,1.3479,1.3381,1.2627,0.9930,0.9076,1.0564,1.0564,1.96,1.08,1.08,0.72,1.60,0.00,0.36,0.36,0.35,1.61,0.39,0.26,0.13,-0.52,0.53,0.80,0.86,2.09,1.87,1.51,-1.27,-0.98,-1.99,-3.32,9,1.310042,0.395000,2.790375,3.555000
53,STOCH_RSI_LinearReg,14,35,9,7,3,20,10,0,1.3229,0.9458,0.7548,0.7739,0.8378,0.8757,0.9270,0.9160,0.9341,1.1330,1.2271,1.2738,1.2512,1.3180,1.5501,1.6039,1.6039,1.4370,1.4303,1.3326,1.1353,1.0657,1.1717,1.1717,1.49,0.80,1.06,0.36,0.17,0.70,0.71,0.78,0.61,2.05,1.54,0.99,1.16,-0.08,0.54,-0.49,0.62,0.94,1.12,1.10,-0.09,-0.67,-1.37,-1.53,14,1.166388,0.521250,2.329425,7.297500
39,STOCH_RSI_LinearReg,14,30,9,7,3,20,12,0,0.7424,0.4846,0.7069,0.8069,1.0096,1.0072,0.9577,0.9658,0.9578,1.1709,1.0257,0.8806,0.8389,1.2090,1.6689,1.6446,1.6446,1.6446,1.6446,1.5722,1.5722,1.5722,1.7532,1.7532,0.91,0.53,1.80,-0.06,-0.24,0.58,1.06,0.15,0.67,2.14,1.87,1.13,-0.24,1.48,0.55,-1.39,-0.75,0.76,1.03,0.90,0.39,-0.33,-0.35,-0.84,9,1.218096,0.489583,1.962862,4.406250
44,STOCH_RSI_LinearReg,14,30,9,7,3,25,12,0,0.7424,0.4846,0.7069,0.8069,1.0096,1.0072,0.9577,0.9658,0.9578,1.1709,1.0257,0.8806,0.8389,1.2090,1.6689,1.6446,1.6446,1.6446,1.6446,1.5722,1.5722,1.5722,1.7532,1.7532,0.91,0.53,1.80,-0.06,-0.24,0.58,1.06,0.15,0.67,2.14,1.87,1.13,-0.24,1.48,0.55,-1.39,-0.75,0.76,1.03,0.90,0.39,-0.33,-0.35,-0.84,9,1.218096,0.489583,1.962862,4.406250
59,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,0.7236,0.6273,0.6094,0.6248,0.8943,1.0515,1.0390,1.0323,1.0648,1.1676,1.1073,1.0486,1.0440,1.1832,1.3877,1.4139,1.4141,1.2774,1.2508,1.1884,1.1144,1.0794,1.1365,1.1365,0.97,0.62,0.04,-0.03,0.23,0.37,0.22,0.02,0.48,1.56,1.04,0.77,0.79,0.65,0.72,0.40,0.42,0.83,0.98,-0.14,-0.24,-0.92,-1.10,-1.26,26,1.067367,0.309167,1.751533,8.038333
28,STOCH_RSI_LinearReg,14,25,9,7,3,25,10,0,10.0000,10.0000,10.0000,1.6786,1.6786,1.4739,1.4739,1.4739,1.4739,1.4739,1.2552,1.0393,0.9702,0.8563,0.6867,0.5451,0.5451,0.5451,0.5451,0.5451,0.5451,0.5451,0.4370,0.4321,2.41,2.71,3.76,3.84,2.78,0.87,-0.50,-0.50,-0.50,-0.51,0.23,-2.87,-3.11,-2.79,-3.67,1.74,2.05,-4.48,-4.36,-5.14,-5.81,-1.84,-3.70,-4.76,1,2.092467,-1.006250,1.092467,-1.006250
23,STOCH_RSI_LinearReg,14,25,9,7,3,20,10,0,10.0000,10.0000,10.0000,1.6786,1.6786,1.

In [2]:
# 4h/1d no LR
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
5,STOCH_RSI,14,25,9,7,3,25,0.1707,0.2336,0.2143,0.2113,0.3284,0.3007,5.1637,4.1494,4.1494,4.1494,3.9756,3.8622,4.0890,4.0890,3.9853,3.9853,3.9853,5.3572,5.3572,5.3572,5.8276,5.8276,6.4067,8.2193,0.79,0.70,0.09,-0.63,1.88,-0.19,42.63,39.38,34.44,26.36,21.96,13.48,40.93,28.36,21.63,37.39,27.67,34.74,23.69,23.30,37.67,36.02,63.68,84.68,2,3.724808,26.693750,5.449617,53.387500
3,STOCH_RSI,14,25,9,7,3,15,0.6295,0.3058,0.2324,0.2233,0.2233,0.1883,1.0887,0.6642,0.6642,0.6642,0.6112,0.5789,0.5789,0.5789,0.5522,0.5522,0.5522,3.0442,3.0442,3.0442,3.8988,3.8988,4.5490,4.6293,0.76,-0.66,-0.43,-2.02,-0.13,-1.76,4.38,-2.16,-3.75,-3.63,-5.26,-5.66,-4.22,-5.93,-6.08,-1.14,-2.38,24.17,5.80,7.14,30.96,27.70,36.12,36.76,1,1.458204,5.357500,0.458204,5.357500
4,STOCH_RSI,14,25,9,7,3,20,0.6295,0.3058,0.2324,0.2233,0.2233,0.1883,1.0887,0.6642,0.6642,0.6642,0.6112,0.5789,0.5789,0.5789,0.5522,0.5522,0.5522,3.0442,3.0442,3.0442,3.8988,3.8988,4.5490,4.6293,0.76,-0.66,-0.43,-2.02,-0.13,-1.76,4.38,-2.16,-3.75,-3.63,-5.26,-5.66,-4.22,-5.93,-6.08,-1.14,-2.38,24.17,5.80,7.14,30.96,27.70,36.12,36.76,1,1.458204,5.357500,0.458204,5.357500
0,STOCH_RSI,14,20,9,7,3,15,0.9749,0.8708,1.1414,1.2908,1.6228,1.6903,1.9473,1.9473,1.5952,1.2122,1.2887,1.2887,1.2887,0.9701,0.8667,0.6245,0.5810,0.5810,0.5810,0.5810,0.5810,0.5810,0.5810,0.5810,0.55,0.18,1.62,2.02,3.37,2.07,3.46,2.62,2.31,2.08,2.17,0.94,1.87,1.39,0.08,-0.53,-0.63,-0.04,0.46,1.56,1.26,1.08,0.27,-0.81,5,1.052850,1.222917,0.264250,6.114583
10,STOCH_RSI,14,35,9,7,3,20,0.5904,0.2740,0.2700,0.4039,0.5166,0.6045,0.9425,1.2167,1.2167,1.2167,1.3928,1.3928,1.3928,1.3928,1.3928,1.4223,1.4512,1.2701,1.2701,1.2701,1.2427,1.2427,1.1594,1.0472,0.47,0.00,-1.15,-0.78,1.40,1.72,2.34,5.16,4.99,4.84,4.22,3.75,3.59,3.43,4.06,4.68,2.50,2.34,2.18,1.72,1.41,1.25,1.56,1.72,3,1.066325,2.391667,0.198975,7.175000
11,STOCH_RSI,14,35,9,7,3,25,0.5904,0.2740,0.2700,0.4039,0.5166,0.6045,0.9425,1.2167,1.2167,1.2167,1.3928,1.3928,1.3928,1.3928,1.3928,1.4223,1.4512,1.2701,1.2701,1.2701,1.2427,1.2427,1.1594,1.0472,0.47,0.00,-1.15,-0.78,1.40,1.72,2.34,5.16,4.99,4.84,4.22,3.75,3.59,3.43,4.06,4.68,2.50,2.34,2.18,1.72,1.41,1.25,1.56,1.72,3,1.066325,2.391667,0.198975,7.175000
9,STOCH_RSI,14,35,9,7,3,15,2.2174,0.4060,0.3968,0.3968,0.5257,0.5704,0.6597,1.0842,1.0842,1.0842,1.3569,1.3569,1.3569,1.3569,1.3569,1.3569,1.4016,1.1481,1.1481,1.1481,1.1126,1.1126,1.0086,0.8774,1.08,0.57,-1.43,-0.96,0.13,0.99,1.58,2.99,2.97,3.19,4.13,2.37,2.54,1.92,1.39,2.34,2.81,0.93,0.32,0.34,-0.20,-0.46,-0.27,-0.82,2,1.063496,1.185417,0.126992,2.370833
6,STOCH_RSI,14,30,9,7,3,15,5.0000,0.7576,0.7576,0.7576,0.7353,0.7353,0.7353,0.7353,0.7353,0.7353,1.3529,1.3529,1.3529,1.3529,1.3529,1.3529,1.3529,0.9020,0.9020,0.9020,0.8519,0.8519,0.7187,0.5750,1.69,1.15,-1.15,-1.15,-1.15,0.27,0.81,0.68,0.95,1.55,3.11,0.68,1.49,0.41,-1.28,0.00,0.00,-0.47,-1.55,-1.35,-1.96,-2.16,-2.09,-3.51,1,1.119100,-0.209583,0.119100,-0.209583
7,STOCH_RSI,14,30,9,7,3,20,5.0000,0.7576,0.7576,0.7576,0.7353,0.7353,0.7353,0.7353,0.7353,0.7353,1.3529,1.3529,1.3529,1.3529,1.3529,1.3529,1.3529,0.9020,0.9020,0.9020,0.8519,0.8519,0.7187,0.5750,1.69,1.15,-1.15,-1.15,-1.15,0.27

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '4h'
higher_timeframe = '1d'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [20]},
                'LinearReg': {'timeperiod': [12], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            